<a href="https://colab.research.google.com/github/arakenufersa/IA/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install graphviz


## **Carregando os dados**

In [ ]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
from google.colab import files
from scipy.io import arff
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import graphviz

# Fazer upload do arquivo ARFF
uploaded = files.upload()

# O nome do arquivo é a chave no dicionário uploaded
filename = next(iter(uploaded))

# Carregar o arquivo ARFF
data, meta = arff.loadarff(filename)

# Converter para DataFrame do pandas
df = pd.DataFrame(data)

# Exibir os primeiros registros do DataFrame
print(df.head())

# Identificar as colunas de atributos e a coluna de classe
# (presumindo que a última coluna seja a coluna alvo, ajuste conforme necessário)
X = df.iloc[:, :-1].values  # Todas as colunas exceto a última (atributos)
y = df.iloc[:, -1].astype(str).values  # Última coluna (classe) convertida para string

# Normalizar os dados (StandardScaler padroniza os dados com média 0 e desvio padrão 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalizando todos os dados antes da validação cruzada

# Exibir os primeiros registros do DataFrame
print(X.view())

## **MLP com holdout com repetições**

In [ ]:
# Definir o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(5,), activation='logistic', learning_rate='constant', learning_rate_init= 0.01, max_iter=1000, random_state=None)

# Inicializar listas para armazenar as acurácias
accuracies = []

# Realizar Holdout com 30 repetições
for i in range(30):
    # Dividir os dados em treino (80%) e teste (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)  # random_state=None para diferentes divisões

    # Treinar o modelo
    mlp.fit(X_train, y_train)

    # Fazer previsões
    y_pred = mlp.predict(X_test)

    # Avaliar a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calcular a média e o desvio padrão das acurácias
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

# Exibir os resultados
print(f"Acurácia média do MLP (Holdout com 30 repetições): {mean_accuracy * 100:.2f}%")
print(f"Desvio padrão da acurácia: {std_accuracy * 100:.2f}%")

## **MLP com validação cruzada**

In [ ]:
# Definir o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(5,), activation='logistic', learning_rate='constant', learning_rate_init= 0.01, max_iter=1000, random_state=None)

# Realizar validação cruzada com 10 grupos
cv_scores = cross_val_score(mlp, X, y, cv=10)

# Calcular a média e o desvio padrão das acurácias
mean_accuracy = np.mean(cv_scores)
std_accuracy = np.std(cv_scores)

# Exibir os resultados
print(f"Acurácia média do MLP (Validação Cruzada com 10 grupos): {mean_accuracy * 100:.2f}%")
print(f"Desvio padrão da acurácia: {std_accuracy * 100:.2f}%")